In [1]:
from aframe import AFrame
import pandas as pd
from IPython.display import display, HTML
from pandas.io.json import json_normalize
import json
from pandas.io.json import json_normalize
import findspark
findspark.init()
import pyspark
from pyspark.sql import Row
from pyspark.sql import SQLContext
from pyspark import SparkContext
import pyspark.sql.functions as func
from pyspark.sql.window import Window

In [2]:
%%HTML
<style type="text/css">
table.dataframe td, table.dataframe th {
    border: 1px solid black;
}

In [3]:
message = AFrame('TinySocial','ChirpMessages')

In [4]:
message.head(2)

,chirpId,messageText,referredTopics,sendTime,senderLocation,user
0,1,love product-z its customization is good:),"[product-z, customization]",2008-04-26T10:10:00.000Z,"[47.44, 80.65]","{'screenName': 'NathanGiesen@211', 'lang': 'en..."
1,10,hate ccast its voice-clarity is OMG:(,"[ccast, voice-clarity]",2008-01-26T10:10:00.000Z,"[29.15, 76.53]","{'screenName': 'ColineGeyer@63', 'lang': 'en',..."


In [5]:
flat_message = message.flatten()
flat_message.head(2)

\
  chirpId                                  messageText   
0       1   love product-z its customization is good:)   
1      10        hate ccast its voice-clarity is OMG:(   

                                                                         \
               referredTopics                  sendTime  senderLocation   
0  [product-z, customization]  2008-04-26T10:10:00.000Z  [47.44, 80.65]   
1      [ccast, voice-clarity]  2008-01-26T10:10:00.000Z  [29.15, 76.53]   

            user                                                     \
  followersCount friendsCount lang           name        screenName   
0          49416        39339   en  Nathan Giesen  NathanGiesen@211   
1          17159          121   en   Coline Geyer    ColineGeyer@63   

                 
  statusesCount  
0           473  
1           362

# Pandas

In [25]:
cols = ['id', 'ename', 'job', 'mgr_id', 'hiredate', 'sal', 'comm', 'deptno']
data = [(1, "SMITH", "CLERK", 8, "17-Dec-80", 800, 20, 10),
(2, "ALLEN", "SALESMAN", 6, "20-Feb-81", 1600, 300, 30),
(3, "WARD", "SALESMAN", 6, "22-Feb-81", 1250, 500, 30),
(4, "JONES", "MANAGER", 9, "2-Apr-81", 2975, 0, 20),
(5, "MARTIN", "SALESMAN", 6, "28-Sep-81", 1250, 1400, 30),
(6, "BLAKE", "MANAGER", 9, "1-May-81", 2850, 0, 30),
(7, "CLARK", "MANAGER", 9, "9-Jun-81", 2450, 0, 10),
(8, "SCOTT", "ANALYST", 4, "19-Apr-87", 3000, 0, 20),
(9, "KING", "PRESIDENT", 0, "17-Nov-81", 5000, 0, 10),
(10, "TURNER", "SALESMAN", 6, "8-Sep-81", 1500, 0, 30),
(11, "ADAMS", "CLERK", 8, "23-May-87", 1100, 0, 20)]

In [26]:
pd_df = pd.DataFrame(data, columns= cols)

In [27]:
pd_df

,id,ename,job,mgr_id,hiredate,sal,comm,deptno
0,1,SMITH,CLERK,8,17-Dec-80,800,20,10
1,2,ALLEN,SALESMAN,6,20-Feb-81,1600,300,30
2,3,WARD,SALESMAN,6,22-Feb-81,1250,500,30
3,4,JONES,MANAGER,9,2-Apr-81,2975,0,20
4,5,MARTIN,SALESMAN,6,28-Sep-81,1250,1400,30
5,6,BLAKE,MANAGER,9,1-May-81,2850,0,30
6,7,CLARK,MANAGER,9,9-Jun-81,2450,0,10
7,8,SCOTT,ANALYST,4,19-Apr-87,3000,0,20
8,9,KING,PRESIDENT,0,17-Nov-81,5000,0,10
9,10,TURNER,SALESMAN,6,8-Sep-81,1500,0,30


In [28]:
pd_df['sal'].sort_values().rolling(window=2).sum()

0        NaN
10    1900.0
2     2350.0
4     2500.0
9     2750.0
1     3100.0
6     4050.0
5     5300.0
3     5825.0
7     5975.0
8     8000.0
Name: sal, dtype: float64

In [10]:
pd_df['sum'] = pd_df['sal'].rolling(window=2).sum()
pd_df

,id,ename,job,mgr_id,hiredate,sal,comm,deptno,sum
0,1,SMITH,CLERK,8.0,17-Dec-80,800,20,10,NaN
1,2,ALLEN,SALESMAN,6.0,20-Feb-81,1600,300,30,2400.0
2,3,WARD,SALESMAN,6.0,22-Feb-81,1250,500,30,2850.0
3,4,JONES,MANAGER,9.0,2-Apr-81,2975,0,20,4225.0
4,5,MARTIN,SALESMAN,6.0,28-Sep-81,1250,1400,30,4225.0
5,6,BLAKE,MANAGER,9.0,1-May-81,2850,0,30,4100.0
6,7,CLARK,MANAGER,9.0,9-Jun-81,2450,0,10,5300.0
7,8,SCOTT,ANALYST,4.0,19-Apr-87,3000,0,20,5450.0
8,9,KING,PRESIDENT,NaN,17-Nov-81,5000,0,10,8000.0
9,10,TURNER,SALESMAN,6.0,8-Sep-81,1500,0,30,6500.0


# Spark

In [11]:
sc = SparkContext(appName="app")
sqlContext = SQLContext(sc)
employee = sqlContext.read.json('/Users/gift/github/Data/employee.json')

In [12]:
employee.columns

['comm', 'deptno', 'ename', 'hiredate', 'id', 'job', 'mgr_id', 'sal']

In [13]:
employee.toPandas().head(2)

,comm,deptno,ename,hiredate,id,job,mgr_id,sal
0,20,10,SMITH,17-Dec-80,1,CLERK,8.0,800
1,300,30,ALLEN,20-Feb-81,2,SALESMAN,6.0,1600


In [14]:
window = Window.partitionBy('mgr_id').orderBy(employee['sal'].desc())

In [15]:
employee.select(func.sum(employee['sal']).over(window)).toPandas()

,sum(sal) OVER (PARTITION BY mgr_id ORDER BY sal DESC NULLS LAST unspecifiedframe$())
0,1100
1,1900
2,5000
3,3000
4,1600
5,3100
6,5600
7,5600
8,2975
9,5825


In [16]:
window = Window.orderBy(employee['sal'].desc()).rowsBetween(-1,0)
employee.withColumn('sal_dif', func.max(employee['sal']).over(window) - employee['sal']).toPandas()

,comm,deptno,ename,hiredate,id,job,mgr_id,sal,sal_dif
0,0,10,KING,17-Nov-81,9,PRESIDENT,NaN,5000,0
1,0,20,SCOTT,19-Apr-87,8,ANALYST,4.0,3000,2000
2,0,20,JONES,2-Apr-81,4,MANAGER,9.0,2975,25
3,0,30,BLAKE,1-May-81,6,MANAGER,9.0,2850,125
4,0,10,CLARK,9-Jun-81,7,MANAGER,9.0,2450,400
5,300,30,ALLEN,20-Feb-81,2,SALESMAN,6.0,1600,850
6,0,30,TURNER,8-Sep-81,10,SALESMAN,6.0,1500,100
7,500,30,WARD,22-Feb-81,3,SALESMAN,6.0,1250,250
8,1400,30,MARTIN,28-Sep-81,5,SALESMAN,6.0,1250,0
9,0,20,ADAMS,23-May-87,11,CLERK,8.0,1100,150


In [17]:
func.max(employee['sal']).over(window) - employee['sal']

Column<b'(max(sal) OVER (ORDER BY sal DESC NULLS LAST ROWS BETWEEN -1 FOLLOWING AND CURRENT ROW) - sal)'>

# AFrame

In [18]:
from aframe import window

In [19]:
af = AFrame(dataverse='testData', dataset='employee')

In [20]:
window = window.Window().orderBy('sal', order='DESC').rowsBetween(1,0)

In [22]:
ordered_af = af.rolling(window, on='sal').sum()

In [23]:
ordered_af.query

'SELECT VALUE sum(t.sal) OVER(ORDER BY sal DESC ROWS BETWEEN 1 PRECEDING AND CURRENT ROW) FROM testData.employee t;'

In [24]:
ordered_af.collect()

,0
0,5000
1,8000
2,5975
3,5825
4,5300
5,4050
6,3100
7,2750
8,2500
9,2350
